# Profitable apps on Google Play Store and App Store for a software development company

# Table of Contents
---
1. [Introduction](#1-introduction)
2. [Data Import](#2-data-import)
    * [Creating the `explore_data()` function](#22-creating-the-explore_data-function)
    * [Testing the `explore_data()` function](#23-testing-the-explore_data-function)
3. [Data Cleaning](#3-data-cleaning)
    * [Deleting Wrong Data](#31-deleting-wrong-data)
    * [Removing Duplicate Entries](#32-removing-duplicate-entries)
    * [Removing non-English apps](#33-removing-non-english-apps)
    * [Isolating the Free Apps](#34-isolating-the-free-apps)
4. [Analysis](#4-analysis)
    * [More Common Apps by Genre](#41-more-common-apps-by-genre)
    * [Most popular Apps by Genre on the iOS App Store](#42-most-popular-apps-by-genre-on-the-ios-app-store)
    * [Most popular Apps by Genre on the Google Play Store](#43-most-popular-apps-by-genre-on-google-play-store)
5. [Conclusions](#5-conclusions)

## 1 Introduction
---

This project's aim is to find mobile apps on Google Play Store and App Store that are more profitable for a software development company. The apps developed by the company are free and hence, gain revenue from the in-app ads. The more the users, the more profit the company acquires. The goal for this project is to analyze the available data to give more insights to the developers on the kind of apps that attract more users.

The figure below shows a distribution of apps available on Google Play Store and App Store, an indication of their popularity:

![img](https://s3.amazonaws.com/dq-content/350/py1m8_statista.png)
Source: [Statista](https://www.statista.com/statistics/276623/number-of-apps-available-in-leading-app-stores/)

The dataset (the source) for this analysis can found in the following links:
* [Google Play data](https://www.kaggle.com/datasets/lava18/google-play-store-apps). Date of collection: August 2018. Number of apps: 10841.
* [App Store data](https://www.kaggle.com/datasets/ramamet4/app-store-apple-data-set-10k-apps). Date of collection: July 2017. Number of apps: 7197.

**Column names and description for Google Play Store dataset**

| Column name | Description |
|:----------|:----------|
| App | Application name |
| Category | Category the app belongs to |
| Rating | Overall user rating of the app (as when scraped) |
| Reviews | Number of user reviews for the app (as when scraped) |
| Size | Size of the app (as when scraped) |
| Installs | Number of user downloads/installs for the app (as when scraped) |
| Type | Paid or Free |
| Price | Price of the app (as when scraped) |
| Content Rating | Age group the app is targeted at - Children / Mature 21+ / Adult |
| Genres | An app can belong to multiple genres (apart from its main category). For eg, a musical family game will belong to Music, Game, Family genres. |
| Last Updated | Date when the app was last updated on Play Store (as when scraped) |
| Current Ver | Current version of the app available on Play Store (as when scraped) |
| Android Ver | Min required Android version (as when scraped) |

Source: [Kaggle](https://www.kaggle.com/datasets/lava18/google-play-store-apps)



**Column names and description for App Store dataset**

| Column name | Description |
|:----------|:----------|
| id | App ID |
| track_name | Application name |
| size_bytes | Memory size (in Bytes) |
| currency | Currency Type |
| price | Price amount |
| rating_count_tot | User Rating counts (for all version) |
| rating_count_ver | User Rating counts (for current version) |
| user_rating | Average User Rating value (for all version) |
| user_rating_ver | Average User Rating value (for current version) |
| ver | Latest version code |
| cont_rating | Content Rating |
| prime_genre | Primary Genre |
| sup_devices.num | Number of supporting devices |
| ipadSc_urls.num | Number of screenshots showed for display |
| lang.num | Number of supported languages |
| vpp_lic | Vpp Device Based Licensing Enabled |

Source: [Kaggle](https://www.kaggle.com/datasets/ramamet4/app-store-apple-data-set-10k-apps?select=AppleStore.csv)


## 2 Data Import
---

### 2.1 Opening and Exploring the data

In [2]:
from csv import reader

## The Google Play Store dataset
gg_opened_file = open('googleplaystore.csv')
gg_read_file = reader(gg_opened_file)
andr = list(gg_read_file)
andr_header = andr[0]
andr_file = andr[1:]

## The App Store dataset
appSt_opened_file = open('AppleStore.csv')
appSt_read_file = reader(appSt_opened_file)
appSt = list(appSt_read_file)
appSt_header = appSt[0]
appSt_file = appSt[1:]

### 2.2 Creating the `explore_data()` function

In [3]:
def explore_data(dataset, start, end, rows_and_columns=False):
    '''
    A function to explore a dataset 
    
    Parameters:
    -----------
        dataset: the dataset to be explored
        start: start index
        end: end index
        rows_and_columns: whether the dataset contains rows and columns or not
    '''
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

### 2.3 Testing the `explore_data()` function

In [4]:
print(appSt_header)
print('\n')
explore_data(appSt_file, 0, 4, True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


Number of rows: 7197
Number of columns: 16


The App Store dataset has 7197 apps and each app has 16 properties. The columns that will be relevant for the analysis are `'track_name'`, `'currency'`, `'price'`, `'rating_count_tot'`, `'rating_count_ver'` and `'prime_genre''`.

In [5]:
print(andr_header)
print('\n')
explore_data(andr_file, 0, 4, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 10841
Number of columns: 13


The Google Play Store dataset has 10841 apps and each app has 13 properties. The columns that will be relevant for the analysis are `'App'`, `'Category'`, `'Reviews'`, `'Installs'`, `'Type'`, `'Price'` and `'Genres'`.

## 3 Data Cleaning
---

### 3.1 Deleting Wrong Data

According to the discussion section of the Google Play dataset on Kaggle, there's a wrong entry on the row, 10472.

In [6]:
print(andr_header)
print('\n')
print(andr_file[10472])

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


The rating for Google Play apps ranges from 1 - 5. But considering the Rating column for the aforementioned row, the app's rating is 19 which is outside the allowed range. 

Therefore, the row, 10472 is going to be deleted.

In [7]:
# Deleting the row with the wrong input 
del andr_file[10472]

### 3.2 Removing Duplicate Entries

Upon exploring the Google Play dataset, it is noticed there are some apps with duplicate entries. Hence, there's the need to work on them.

In [8]:
# For instance, the Instagram app
for app in andr_file:
    name = app[0]       # Select the app name
    if name == 'Instagram':
        print(app)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


In [9]:
# Check the number of apps that have duplicate entries
duplicate_apps = []
unique_apps = []

for app in andr_file:
    name = app[0]               # Select the app name
    if name in unique_apps:     # If the app is already found in the 'unique_apps' list
        duplicate_apps.append(name) # Then it's a duplicate entry
    else:
        unique_apps.append(name)    # Else it's unique
        
print('Number of duplicate apps:', len(duplicate_apps))
print('\n')
print('Examples of duplicate apps are ', duplicate_apps[:10])

Number of duplicate apps: 1181


Examples of duplicate apps are  ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack']


The duplicate rows of the apps can't be removed randomly since the most current record of the app might be deleted. The entry among the duplicates with the most reviews indicate the most recent record of the app in question. Therefore, for each app with duplicate entries, the row with the **highest** `Reviews` will be selected to represent the app.

In [10]:
# Selecting the apps with the highest reviews
reviews_max = {}

for app in andr_file:
    name = app[0]               # Select the app name
    n_reviews = float(app[3])   # Select the user reviews of the app and convert it to a float
    
    # If the app is already in the 'reviews_max' dictionary and the associated user reviews
    # is less than the new user reviews
    if name in reviews_max and reviews_max[name] < n_reviews:   
        reviews_max[name] = n_reviews   # Update the user reviews with the new one
    elif name not in reviews_max:  
        reviews_max[name] = n_reviews

In [11]:
# Validating the 'reviews_max' dictionary
print('Expected length: ', len(andr_file) - 1181)
print('Actual length: ', len(reviews_max))

Expected length:  9659
Actual length:  9659


The next step is to use the `reviews_max` dictionary to clean the dataset. To deal with the duplicates, the entries with the highest reviews will be kept.

In [12]:
# Initialization of the lists to store the duplicate-free dataset and 
# the apps that are already added to the list of duplicate-free dataset
android_clean = []
already_added = []

for app in andr_file:
    name = app[0]               # Select the app name
    n_reviews = float(app[3])   # Select the user reviews of the app and convert it to a float
    
    if n_reviews == reviews_max[name] and name not in already_added:
        android_clean.append(app)
        already_added.append(name)

In [13]:
# Explore the duplicate-free dataset to find out if its length
# is equal to that of the 'review_max' dictionary
explore_data(android_clean, 0, 5, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', '50,000+', 'Free', '0', 'Everyone', 'Art & Design', 'March 26, 2017', '1.0', '2.3 and up']


Number of rows: 9659
Number of columns: 13


The output signifies that the length of the duplicate-free dataset is **`9659`** which is equal to that of the `review_max` dictionary. This implies the new dataset is rid of duplicate entries.

### 3.3 Removing non-English apps

Exploring the apps further shows that the two datasets contain non-English apps. Since the company targets its English-speaking customers, non-English apps will be of no use to such customers. Consequently, there's the need to delete such apps from the datasets.

The function below is to execute that task:

In [14]:
def is_English_app(app_name):
    '''
    A function to check whether an app name is in English or not. This is 
    done by looping through the characters of the app name. If 3 or more 
    characters are greater than 127 - the upper bound of the ASCII code 
    range for English characters, then the app name is not in English. 
    Otherwise, it is in English.
    
    Parameter:
    ----------
        app_name: the name of the app
        
    Return:
    -------
        A boolean
    '''
    count = 0
    for character in app_name:
        if ord(character) > 127:
            count += 1
        
    if count > 3:
        return False
    
    return True
    

In [15]:
# Testing the 'is_English_app' function
is_English_app('Instagram')

True

In [16]:
is_English_app('爱奇艺PPS -《欢乐颂2》电视剧热播')

False

In [17]:
is_English_app('Docs To Go™ Free Office Suite')

True

In [18]:
is_English_app('Instachat 😜')

True

In [19]:
# Filter out non-English apps for the Google Play and iOS App datasets
android_English = []
iOS_English = []

# Android
for app in android_clean:
    name = app[0]
    if is_English_app(name):
        android_English.append(app)

# iOS
for app in appSt_file:
    name = app[1]
    if is_English_app(name):
        iOS_English.append(app)
        
explore_data(android_English, 0, 5, True)
print('\n')
explore_data(iOS_English, 0, 5, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', '50,000+', 'Free', '0', 'Everyone', 'Art & Design', 'March 26, 2017', '1.0', '2.3 and up']


Number of rows: 9614
Number of columns: 13


['284882215', 'Facebook', '389879808', 'USD', 

### 3.4 Isolating the Free Apps

Since the company builds only free apps and gain their revenue from in-app ads, it is necessary to select only the free apps for the analysis.

In [20]:
# Selecting the free apps in the Google Play Store dataset
android_Eng_Final = []
for app in android_English:
    price = app[7]      # Select the price of the app
    if price == '0':    # If it is free
        android_Eng_Final.append(app)
        
# Selecting the free apps in the iOS App Store dataset
iOS_Eng_Final = []
for app in iOS_English:
    price = app[4]      # Select the price of the app
    if price == '0.0':  # If it is free
        iOS_Eng_Final.append(app)
        
        
# Check the outputs
explore_data(android_Eng_Final, 0, 3, True)
print('\n')
explore_data(iOS_Eng_Final, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 8864
Number of columns: 13


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 

There are `8864` **Android apps** and `3222` **iOS apps** remaining after the data cleaning steps.

## 4 Analysis
---

### 4.1 More Common Apps by Genre

As mentioned in the introduction, the aim is to determine the kinds of apps that are likely to attract more users because our revenue is highly influenced by the number of people using our apps.

Since the goal is to add the app on both platforms - iOS App Store and Google Play Store, it is necessary to find the app profiles that are successful on both markets. For example, a profile that might work well for both markets might be a productivity app that makes use of gamification.

To commence the analysis, it is important to get a sense of the most common genres for each market. For this, frequency tables for the **`Genres`** and **`Category`** columns of the Google Play Store data set and the **`prime_genre`** column of the iOS App Store data set are generated. To implement this, two functions are created:

In [21]:
def freq_table(dataset, index):
    '''
    A function to create a frequency table that shows percentages.
    
    Parameter:
    ----------
        dataset: A list of lists
        index: An integer representing a column index
        
    Return:
    -------
        A dictionary of percentages
    '''
    table = {}
    total = 0
    
    # Create a frequency table
    for app in dataset:
        total += 1
        value = app[index]
        if value in table:
            table[value] += 1
        else:
            table[value] = 1

    # Convert frequencies to percentages        
    table_percentages = {}
    for key in table:
        percentage = (table[key] / total) * 100
        table_percentages[key] = percentage
        
    return table_percentages


def display_table(dataset, index):
    '''
    A function to generate a frequency table, transform the table into 
    a  list of tuples and sort the list in a descending order.
    
    Parameters:
    -----------
        dataset: A list of lists
        index: An integer representing a column index
    '''
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])
    

First, the frequency table for the `prime_genre` column of the **iOS App Store** dataset is examined.

In [22]:
display_table(iOS_Eng_Final, -5)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


From the above results, it is noticed among the free English apps that:
* over **58.16%** of the apps are `games` which is more than half. 
* `Entertainment` apps are approximately **8%** of the total apps. 
* `photo and video` apps, which are approximately **5%**. 
* Only **3.66%** of the apps are designed for `education`.
* followed by `social networking` apps which is **3.29%** of the apps in our data set.

The implication is that apps designed for fun (games, entertainment, photo and video, social networking, sports, music, etc.) dominates the iOS App Store (at least the part containing free English apps), while apps with practical purposes (education, shopping, utilities, productivity, lifestyle, etc.) are more rare. However, the fact that fun apps are the most numerous doesn't also imply that they also have the greatest number of users — the demand might not be the same as the offer.

Next, a similar analysis is done for the **Google Play Store** dataset by examining the `Genres` and `Category` columns.

In [23]:
display_table(android_Eng_Final, 1) # Category column

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

The case is quite different with the Google Play Store dataset. There are not so many apps designed for fun but it seems that a sizable number of apps are designed for practical purposes (family, tools, business, lifestyle, productivity, etc.). Nonetheless, upon further exploration, it is recognised that the family category (which accounts for almost 19% of the apps) usually means games for kids.

That notwithstanding, apps for practical purposes seem to have a better representation on Google Play Store compared to App Store. This is also confirmed by the frequency table that is generated for the Genres column.

In [24]:
display_table(android_Eng_Final, -4) # Genres column

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

Up until now, it is found that apps designed for fun dominates the App Store, while Google Play portrays a more balanced landscape of both fun and practical apps.

The next step is to find the kind of apps that have most users.

### 4.2 Most popular Apps by Genre on the iOS App Store

One way to find out what genres are the most popular (have the most users) is to calculate the average number of installs for each app genre. For the Google Play data set, this information can be found in the `Installs` column, but for the iOS App Store data set this information is missing. As a workaround, the total number of user ratings will be used as a proxy, which can be found in the `rating_count_tot` column.

Below, the average number of user ratings per app genre on the App Store is calculated:

In [25]:
genres_ios = freq_table(iOS_Eng_Final, -5)

for genre in genres_ios:
    total = 0
    len_genre = 0
    
    for app in iOS_Eng_Final:
        genre_app = app[-5]
        if genre_app == genre:
            num_ratings = float(app[5])
            total += num_ratings
            len_genre += 1
    
    avg_num_ratings = total / len_genre
    print(genre, ':', avg_num_ratings)

Social Networking : 71548.34905660378
Photo & Video : 28441.54375
Games : 22788.6696905016
Music : 57326.530303030304
Reference : 74942.11111111111
Health & Fitness : 23298.015384615384
Weather : 52279.892857142855
Utilities : 18684.456790123455
Travel : 28243.8
Shopping : 26919.690476190477
News : 21248.023255813954
Navigation : 86090.33333333333
Lifestyle : 16485.764705882353
Entertainment : 14029.830708661417
Food & Drink : 33333.92307692308
Sports : 23008.898550724636
Book : 39758.5
Finance : 31467.944444444445
Education : 7003.983050847458
Productivity : 21028.410714285714
Business : 7491.117647058823
Catalogs : 4004.0
Medical : 612.0


Looking at the output above, navigation apps have the highest number of user reviews. But this outcome is greatly swayed by apps such as Google Maps and Waze. This is proven below:

In [26]:
for app in iOS_Eng_Final:
    if app[-5] == 'Navigation':
        print(app[1], ':', app[5]) # The name and ratings

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


A similar situation applies to social networking apps, where the average number is heavily impacted by a few giants like Pinterest, Skype,  Instagram, Facebook, etc. The same pattern applies to music apps, where a few big players like Spotify, Shazam, and Pandora greatly influence the average number.

The goal is to find popular genres, but the popularity of navigation, social networking or music apps might seem apparent. The average number of ratings seem to be distorted by very few apps which have hundreds of thousands of user ratings, while the other apps may struggle to get past the 10,000 threshold. 

Reference apps have an average user ratings of 74,942 which is also skewed by the average ratings of Bible and Dictionary.com. The proof is depicted below:

In [27]:
for app in iOS_Eng_Final:
    if app[-5] == 'Reference':
        print(app[1], ':', app[5]) # The name and ratings

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
WWDC : 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
VPN Express : 14
Real Bike Traffic Rider Virtual Reality Glasses : 8
教えて!goo : 0
Jishokun-Japanese English Dictionary & Translator : 0


A recommendation to be considered is to take another popular book and turn it into an app where we could add different features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes about the book, etc. In addition, we could also embed a dictionary within the app to increase its usability.

This idea seems to fit well with the fact that the App Store is dominated by fun apps. This suggests the market might be a bit saturated with for-fun apps, which means a practical app might have more of a chance to stand out among the huge number of apps on the App Store.

Other genres that seem popular include weather, food and drink, or finance. But they are unappealing to the company because more cost of production has to be incurred for additional and external services outside the scope of the company.

### 4.3 Most Popular Apps by Genre on Google Play Store

For this market, the number of installs for each app is available which can give an overview of an app's popularity. Nevertheless, the figures for installs are inexact:

In [28]:
display_table(android_Eng_Final, 5) # The column for installs

1,000,000+ : 15.726534296028879
100,000+ : 11.552346570397113
10,000,000+ : 10.548285198555957
10,000+ : 10.198555956678701
1,000+ : 8.393501805054152
100+ : 6.915613718411552
5,000,000+ : 6.825361010830325
500,000+ : 5.561823104693141
50,000+ : 4.7721119133574
5,000+ : 4.512635379061372
10+ : 3.5424187725631766
500+ : 3.2490974729241873
50,000,000+ : 2.3014440433213
100,000,000+ : 2.1322202166064983
50+ : 1.917870036101083
5+ : 0.78971119133574
1+ : 0.5076714801444043
500,000,000+ : 0.2707581227436823
1,000,000,000+ : 0.22563176895306858
0+ : 0.04512635379061372
0 : 0.01128158844765343


But for the case at hand, there is no need for perfect precision with respect to the number of users. Hence, the numbers are going to be left as they are, which means that it will be considered that an app with 100,000+ installs has 100,000 installs, and an app with 1,000,000+ installs has 1,000,000 installs, and so on.

To perform computations, it will be necessary to convert each install number to float. This means that the commas and the plus characters need to be removed, otherwise the conversion will fail and raise an error.

In [29]:
android_categories = freq_table(android_Eng_Final, 1)

for category in android_categories:
    total = 0
    len_category = 0
    
    for app in android_Eng_Final:
        category_app = app[1]
        if category_app == category:
            num_installs = app[5]
            num_installs = num_installs.replace(',', '')    # Removing the commas
            num_installs = num_installs.replace('+', '')    # Removing the plus characters
            total += float(num_installs)
            len_category += 1
    avg_num_installs = total / len_category
    print(category, ':', avg_num_installs)

ART_AND_DESIGN : 1986335.0877192982
AUTO_AND_VEHICLES : 647317.8170731707
BEAUTY : 513151.88679245283
BOOKS_AND_REFERENCE : 8767811.894736841
BUSINESS : 1712290.1474201474
COMICS : 817657.2727272727
COMMUNICATION : 38456119.167247385
DATING : 854028.8303030303
EDUCATION : 1833495.145631068
ENTERTAINMENT : 11640705.88235294
EVENTS : 253542.22222222222
FINANCE : 1387692.475609756
FOOD_AND_DRINK : 1924897.7363636363
HEALTH_AND_FITNESS : 4188821.9853479853
HOUSE_AND_HOME : 1331540.5616438356
LIBRARIES_AND_DEMO : 638503.734939759
LIFESTYLE : 1437816.2687861272
GAME : 15588015.603248259
FAMILY : 3695641.8198090694
MEDICAL : 120550.61980830671
SOCIAL : 23253652.127118643
SHOPPING : 7036877.311557789
PHOTOGRAPHY : 17840110.40229885
SPORTS : 3638640.1428571427
TRAVEL_AND_LOCAL : 13984077.710144928
TOOLS : 10801391.298666667
PERSONALIZATION : 5201482.6122448975
PRODUCTIVITY : 16787331.344927534
PARENTING : 542603.6206896552
WEATHER : 5074486.197183099
VIDEO_PLAYERS : 24727872.452830188
NEWS_AND_

On average, `communication apps` have the most installs: 38,456,119. This number is heavily skewed up by a few apps that have over one billion installs (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail, and Hangouts), and a few others with over 100 and 500 million installs:

In [30]:
for app in android_Eng_Final:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+' 
                                      or app[5] == '500,000,000+' 
                                      or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

If all the communication apps that have over 100 million installs are removed, the average would be reduced roughly ten times:

In [31]:
under_100_mil = []

for app in android_Eng_Final:
    num_installs = app[5]
    num_installs = num_installs.replace(',', '')    # Removing the commas
    num_installs = num_installs.replace('+', '')    # Removing the plus characters
    if (app[1] == 'COMMUNICATION') and (float(num_installs) < 100000000):
        under_100_mil.append(float(num_installs))
        
print("Average installs under 100 million: ", sum(under_100_mil) / len(under_100_mil))

Average installs under 100 million:  3603485.3884615386


The same pattern is seen for the video players category, which is the runner-up with 24,727,872 installs. The market is dominated by apps like Youtube, Google Play Movies & TV, or MX Player. The pattern is repeated for social apps (where we have giants like Facebook, Instagram, Google+, etc.), photography apps (Google Photos and other popular photo editors), or productivity apps (Microsoft Word, Dropbox, Google Calendar, Evernote, etc.).

Again, the main concern is that these app genres might seem more popular than they really are. Moreover, these niches seem to be dominated by a few giants who are hard to compete against.

The game genre seems pretty popular, but it was found previously that this part of the market seems a bit saturated, so a different app recommendation is crucial if possible.

The books and reference genre looks fairly popular as well, with an average number of installs of 8,767,811. It's interesting to explore this in more depth, since it is found that this genre has some potential to work well on the App Store, and the aim is to recommend an app genre that shows potential for being profitable on both the App Store and Google Play.

The next step is to take a look at some of the apps from this genre and their number of installs:

In [32]:
for app in android_Eng_Final:
    if app[1] == 'BOOKS_AND_REFERENCE':
        print(app[0], ':', app[5])

E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Ebook Reader : 5,000,000+
Litnet - E-books : 100,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
E

The book and reference genre includes a variety of apps: software for processing and reading ebooks, various collections of libraries, dictionaries, tutorials on programming or languages, etc. It seems there's still a small number of extremely popular apps that skew the average:

In [33]:
for app in android_Eng_Final:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000,000+'
                                            or app[5] == '500,000,000+'
                                            or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

Google Play Books : 1,000,000,000+
Bible : 100,000,000+
Amazon Kindle : 100,000,000+
Wattpad 📖 Free Books : 100,000,000+
Audiobooks from Audible : 100,000,000+


However, it looks like there are only a few very popular apps, so this market still shows potential. Let's try to get some app ideas based on the kind of apps that are somewhere in the middle in terms of popularity (between 1,000,000 and 100,000,000 downloads):

In [34]:
for app in android_Eng_Final:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000+'
                                            or app[5] == '5,000,000+'
                                            or app[5] == '10,000,000+'
                                            or app[5] == '50,000,000+'):
        print(app[0], ':', app[5])

Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
AlReader -any text book reader : 5,000,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
ReadEra – free ebook reader : 1,000,000+
Ebook Reader : 5,000,000+
Read books online : 5,000,000+
eBoox: book reader fb2 epub zip : 1,000,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
Moon+ Reader : 10,000,000+
English-Myanmar Dictionary : 1,000,000+
Golden Dictionary (EN-AR) : 1,000,000+
All Language Translator Free : 1,000,000+
Aldiko Book Reader : 10,000,000+
Dictionary - WordWeb : 5,000,000+
50000 Free eBooks & Free AudioBooks : 5,000,000+
Al-Quran (Free) : 10,000,000+
Al Quran Indonesia : 10,000,000+
Al'Quran Bahasa Indonesia : 10,000,000+
Al Quran Al karim : 1,000,000+
Al Quran : EAlim - Translations & MP3 Offline : 5,000,000+
Koran Read &MP3 30 Juz Offline : 1,000,000+
H

This niche seems to be dominated by software for processing and reading ebooks, as well as various collections of libraries and dictionaries, so it's probably not a good idea to build similar apps since there'll be some significant competition.

It is also noticed that there are quite a few apps built around the book Quran, which suggests that building an app around a popular book can be profitable. It seems that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the iOS App Store markets.

However, it looks like the market is already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.

## 5 Conclusions

In this project, the data about the iOS App Store and Google Play mobile apps were analyzed, with the aim of recommending an app profile that can be profitable for both markets.

It can be concluded that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets. The markets are already full of libraries, so it is required to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.